In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
ner_features = dataset['train'].features['ner_tags']
ner_features

In [ ]:
label_names = ner_features.feature.names
label_names

In [ ]:
print(len(label_names))

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
tokenizer.is_fast

In [ ]:
inputs = tokenizer(dataset['train'][0]['tokens'], is_split_into_words=True)
inputs.tokens()

In [ ]:
inputs = tokenizer(dataset['train'][0]['tokens'], is_split_into_words=True, truncation=True, padding="max_length", max_length=32, return_tensors='pt')
inputs.tokens()

In [ ]:
inputs.word_ids()

In [ ]:
inputs

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            new_labels.append(label)
    return new_labels

In [ ]:
labels = dataset["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=False, max_length=256
    )

    all_labels = examples['ner_tags']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
from transformers import DataCollatorForTokenClassification
collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset["train"].column_names)
tokenized_datasets["train"].set_format("pt")
tokenized_datasets["validation"].set_format("pt")
tokenized_datasets["test"].set_format("pt")

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets['train'][0]

In [ ]:
tokenized_datasets['validation'][0]

In [ ]:
tokenized_datasets['test'][0]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True, collate_fn=collator)

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
batch['input_ids'].shape

In [ ]:
batch

In [ ]:
for i in range(2):
    print(dataset['train'][i]['tokens'])
    print(tokenizer(dataset['train'][i]['tokens'], is_split_into_words=True, truncation=True).tokens())
    print(tokenized_datasets["train"][i]["labels"])
    print(tokenized_datasets["train"][i]["input_ids"])

In [ ]:
from transformers import AutoModel
encoder = AutoModel.from_pretrained("xlm-roberta-base", add_pooling_layer=False)

In [ ]:
test = dict()
test['input_ids'] = batch['input_ids']
test['attention_mask'] = batch['attention_mask']
output = encoder(**test)
print(output)

In [ ]:
output.last_hidden_state.shape

In [ ]:
import torch

dense = torch.nn.Linear(768, len(label_names))

In [ ]:
res = dense(output.last_hidden_state)
res

In [ ]:
res.shape

In [ ]:
res

In [ ]:
batch['labels']

In [ ]:
batch['labels'].shape

In [ ]:
import torch.nn.functional as F

loss = F.cross_entropy(res.permute(0,2,1), batch['labels'])
loss

In [ ]:
x, y = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}, batch['labels']
x, y